<a href="https://colab.research.google.com/github/CharKuehl/DemoRepository/blob/transpose_vis/AutoML_DermalVOCs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data read in and tidying of Bacteria data



## First steps to read in data:
1.   Read in the bacteria data from link
2.   Put correct headers in place 
3.   Get rid of rows with out compound names
4.   Zero all remaining NaN




In [1]:
bacteria_url = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/FrontiersDS.csv"

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
import torch
import seaborn as sns
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from matplotlib.colors import ListedColormap

In [3]:
# Load ML Pkgs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [48]:
df_rawbac = pd.read_csv(bacteria_url, header=2)
df_rawbac = df_rawbac.iloc[:,0:]
#df_rawbac = df_rawbac.drop(df_rawbac.index[:1]) #drops row 0, because it is NaN
df_rawbac = df_rawbac.dropna(subset=['Compounds'])
print(df_rawbac)

                      Compounds          Chemical_class  ...  EC.B_TSB_D  EC.B_TSB_E
0                 Ethyl Acetate                   Ester  ...    548442.0    339404.0
1                       Ethanol                 Alcohol  ...   3588158.0   3962927.0
2   Propanoic acid, ethyl ester  Fatty acid ethyl ester  ...         0.0         0.0
3                   2-Pentanone                  Ketone  ...    637563.0   2988409.0
4                        Decane                  Alkane  ...         0.0    348154.0
..                          ...                     ...  ...         ...         ...
62              n-Tridecan-1-ol                 Alcohol  ...         0.0         0.0
63               1-Tetradecanol                 Alcohol  ...    146349.0    635257.0
64               n-Pentadecanol                 Alcohol  ...         0.0         0.0
65                1-Hexadecanol                 Alcohol  ...         0.0    482026.0
66                       Indole                  Indole  ...  852

## Clean data frame

In [53]:
df_bac = df_rawbac.fillna(0)
print(df_bac)

                      Compounds          Chemical_class  ...  EC.B_TSB_D  EC.B_TSB_E
0                 Ethyl Acetate                   Ester  ...    548442.0    339404.0
1                       Ethanol                 Alcohol  ...   3588158.0   3962927.0
2   Propanoic acid, ethyl ester  Fatty acid ethyl ester  ...         0.0         0.0
3                   2-Pentanone                  Ketone  ...    637563.0   2988409.0
4                        Decane                  Alkane  ...         0.0    348154.0
..                          ...                     ...  ...         ...         ...
62              n-Tridecan-1-ol                 Alcohol  ...         0.0         0.0
63               1-Tetradecanol                 Alcohol  ...    146349.0    635257.0
64               n-Pentadecanol                 Alcohol  ...         0.0         0.0
65                1-Hexadecanol                 Alcohol  ...         0.0    482026.0
66                       Indole                  Indole  ...  852

## Transposed data frame

In [83]:
tbac = df_bac.transpose()
tbac.reset_index(drop=False, inplace=True)
tbac.columns = tbac.iloc[0]
tbac = tbac.drop(tbac.index[0:4])
tbac.reset_index(drop=True, inplace=True)
tbac

,Compounds,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,1-Propanol,Methyl thiolacetate,"Butanoic acid, 2-methyl-, ethyl ester",2-Hexanone,Ethyl isocyanide,"1-Propanol, 2-methyl-","2-Pentanol, 2-methyl-",2-Pentanol,"1-Butanol, 3-methyl-, acetate",1 - Undecene,1-Butanol,2-Heptanone,Dodecane,"1-Butanol, 3-methyl-",S-Methyl 3-methylbutanethioate,"2-Heptanone, 4,6-dimethyl-","3-Buten-1-ol, 3-methyl-","Thiocyanic acid, methyl ester",Acetoin,"1-Pentanol, 2-methyl-","Butanoic acid, 3-methyl-, 2-methylbutyl ester","2-Heptanol, 4-methyl-",2-Nonanone,Acetic acid,2-Nonanol,Pyrrole,"1H-Pyrrole, 2-methyl-","1-Heptanol, 2,4-dimethyl-,",Propanoic acid,"Propanoic acid, 2-methyl-",1-Octanol,2-Undecanone,"Benzoic acid, methyl ester",Butyrolactone,"Decanoic acid, ethyl ester","Acetic acid, decyl ester",2-Undecanol,"Butanoic acid, 3-methyl-",2-Dodecanone,1-Decanol,2-Tridecanone,"Dodecanoic acid, ethyl ester","1,4-Butanediol",Phenylethyl Alcohol,"Acetophenone, 2'-amino-",2-Tridecanol,Tetradecanal,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,SA.A_TSB_A,465374,1.02772e+06,0,1.28965e+06,800581,324424,73015,0,0,0,0,82772,1.01966e+06,0,0,139019,0,0,111465,0,0,56603,0,3.09769e+06,0,0,334841,0,375792,0,0,0,0,4.01623e+06,0,0,0,0,51090,76990,0,0,0,0,0,0,0,4.00719e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SA.A_TSB_B,193151,1.05097e+06,0,504113,294680,189630,0,0,0,0,0,0,245397,0,0,219733,0,56594,0,0,0,0,0,2.13091e+06,0,0,153294,0,102975,0,0,0,0,1.56558e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,1.691e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SA.A_TSB_C,403286,1.85039e+06,0,1.1695e+06,15,228163,73558,0,0,0,0,0,494407,0,0,196279,0,124321,0,0,0,85741,0,2.76959e+06,0,0,341899,0,173956,0,0,0,0,1.95285e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,2.14015e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,SA.A_TSB_D,129833,5.14077e+06,0,1.92607e+06,124282,0,188367,0,0,0,0,0,2.1985e+06,0,0,375976,197683,53549,0,0,0,0,0,5.10148e+06,0,0,86222,0,381264,0,0,0,0,1.18756e+07,0,0,0,0,111819,0,0,0,0,0,0,0,0,4.67842e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SA.A_TSB_E,117105,3.42256e+06,0,246751,0,0,0,0,0,0,0,0,960937,0,0,93132,66816,0,0,0,0,0,0,2.34986e+06,0,0,0,0,233731,0,0,0,0,1.10244e+07,0,0,0,0,90520,0,0,0,0,0,0,0,0,3.21933e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,EC.B_TSB_A,119313,1.36455e+06,0,169495,0,0,0,0,0,0,0,0,0,0,0,54155,0,0,0,0,934149,151941,0,1.92851e+06,0,0,0,0,0,0,0,0,985092,2.3731e+06,0,0,0,0,0,0,265324,156741,82607,0,0,0,119773,0,0,410056,0,0,0,76521,0,0,0,289615,0,0,0,0,0,0,0,0,6.03256e+07
80,EC.B_TSB_B,128397,1.45114e+06,0,359499,0,0,0,0,0,0,0,0,0,0,0,54486,0,0,0,0,526119,185916,0,2.047e+06,0,0,0,0,0,0,0,0,617890,2.28252e+06,0,0,0,0,0,0,558260,299459,130593,0,0,0,235221,0,0,1.11834e+06,67510,0,0,125495,0,52814,0,749942,0,0,0,0,0,78672,0,0,1.0788e+08
81,EC.B_TSB_C,309004,5.65081e+06,0,875905,0,0,109170,0,0,0,310852,0,0,97651,0,180005,62168,0,0,0,809747,352271,0,6.49783e+06,0,0,92340,0,52741,58804,0,0,481673,8.92709e+06,0,0,0,97887,0,0,464549,362678,136519,0,0,0,360779,0,0,855428,69218,0,0,156776,0,0,0,479315,0,0,0,0,0,84426,0,0,7.21024e+07
82,EC.B_TSB_D,548442,3.58816e+06,0,637563,0,0,60834,0,0,0,483269,0,0,77088,0,114343,0,0,0,0,705595,244700,0,4.14133e+06,0,0,68147,0,85445,0,0,0,610883,6.38814e+06,0,0,0,80664,0,0,1.20103e+06,303087,162366,0,0,0,253951,0,0,2.40514e+06,169862,0,0,165001,0,84957,0,2.75081e+06,0,0,0,0,0,146349,0,0,8.52575e+07


## Visualization data frames

In [68]:
df_t_bac = df_bac.T
new_header = df_t_bac.iloc[0] #grab the first row for the header
df_t_bac = df_t_bac[1:] #take the data less the header row
df_t_bac.columns = new_header #set the header row as the df header

df_t_bac = df_t_bac.drop(df_t_bac.index[0:3])
df_t_bac = df_t_bac.reset_index()
df_t_bac.head(3)

Compounds,index,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,1-Propanol,Methyl thiolacetate,"Butanoic acid, 2-methyl-, ethyl ester",2-Hexanone,Ethyl isocyanide,"1-Propanol, 2-methyl-","2-Pentanol, 2-methyl-",2-Pentanol,"1-Butanol, 3-methyl-, acetate",1 - Undecene,1-Butanol,2-Heptanone,Dodecane,"1-Butanol, 3-methyl-",S-Methyl 3-methylbutanethioate,"2-Heptanone, 4,6-dimethyl-","3-Buten-1-ol, 3-methyl-","Thiocyanic acid, methyl ester",Acetoin,"1-Pentanol, 2-methyl-","Butanoic acid, 3-methyl-, 2-methylbutyl ester","2-Heptanol, 4-methyl-",2-Nonanone,Acetic acid,2-Nonanol,Pyrrole,"1H-Pyrrole, 2-methyl-","1-Heptanol, 2,4-dimethyl-,",Propanoic acid,"Propanoic acid, 2-methyl-",1-Octanol,2-Undecanone,"Benzoic acid, methyl ester",Butyrolactone,"Decanoic acid, ethyl ester","Acetic acid, decyl ester",2-Undecanol,"Butanoic acid, 3-methyl-",2-Dodecanone,1-Decanol,2-Tridecanone,"Dodecanoic acid, ethyl ester","1,4-Butanediol",Phenylethyl Alcohol,"Acetophenone, 2'-amino-",2-Tridecanol,Tetradecanal,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,SA.A_TSB_A,465374,1.02772e+06,0,1.28965e+06,800581,324424,73015,0,0,0,0,82772,1.01966e+06,0,0,139019,0,0,111465,0,0,56603,0,3.09769e+06,0,0,334841,0,375792,0,0,0,0,4.01623e+06,0,0,0,0,51090,76990,0,0,0,0,0,0,0,4.00719e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SA.A_TSB_B,193151,1.05097e+06,0,504113,294680,189630,0,0,0,0,0,0,245397,0,0,219733,0,56594,0,0,0,0,0,2.13091e+06,0,0,153294,0,102975,0,0,0,0,1.56558e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,1.691e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SA.A_TSB_C,403286,1.85039e+06,0,1.1695e+06,15,228163,73558,0,0,0,0,0,494407,0,0,196279,0,124321,0,0,0,85741,0,2.76959e+06,0,0,341899,0,173956,0,0,0,0,1.95285e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,2.14015e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
df_t_bac.rename(columns={"Compounds": "index", "index": "bacteria_title"})

Compounds,bacteria_title,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,1-Propanol,Methyl thiolacetate,"Butanoic acid, 2-methyl-, ethyl ester",2-Hexanone,Ethyl isocyanide,"1-Propanol, 2-methyl-","2-Pentanol, 2-methyl-",2-Pentanol,"1-Butanol, 3-methyl-, acetate",1 - Undecene,1-Butanol,2-Heptanone,Dodecane,"1-Butanol, 3-methyl-",S-Methyl 3-methylbutanethioate,"2-Heptanone, 4,6-dimethyl-","3-Buten-1-ol, 3-methyl-","Thiocyanic acid, methyl ester",Acetoin,"1-Pentanol, 2-methyl-","Butanoic acid, 3-methyl-, 2-methylbutyl ester","2-Heptanol, 4-methyl-",2-Nonanone,Acetic acid,2-Nonanol,Pyrrole,"1H-Pyrrole, 2-methyl-","1-Heptanol, 2,4-dimethyl-,",Propanoic acid,"Propanoic acid, 2-methyl-",1-Octanol,2-Undecanone,"Benzoic acid, methyl ester",Butyrolactone,"Decanoic acid, ethyl ester","Acetic acid, decyl ester",2-Undecanol,"Butanoic acid, 3-methyl-",2-Dodecanone,1-Decanol,2-Tridecanone,"Dodecanoic acid, ethyl ester","1,4-Butanediol",Phenylethyl Alcohol,"Acetophenone, 2'-amino-",2-Tridecanol,Tetradecanal,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,SA.A_TSB_A,465374,1.02772e+06,0,1.28965e+06,800581,324424,73015,0,0,0,0,82772,1.01966e+06,0,0,139019,0,0,111465,0,0,56603,0,3.09769e+06,0,0,334841,0,375792,0,0,0,0,4.01623e+06,0,0,0,0,51090,76990,0,0,0,0,0,0,0,4.00719e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SA.A_TSB_B,193151,1.05097e+06,0,504113,294680,189630,0,0,0,0,0,0,245397,0,0,219733,0,56594,0,0,0,0,0,2.13091e+06,0,0,153294,0,102975,0,0,0,0,1.56558e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,1.691e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SA.A_TSB_C,403286,1.85039e+06,0,1.1695e+06,15,228163,73558,0,0,0,0,0,494407,0,0,196279,0,124321,0,0,0,85741,0,2.76959e+06,0,0,341899,0,173956,0,0,0,0,1.95285e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,2.14015e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,SA.A_TSB_D,129833,5.14077e+06,0,1.92607e+06,124282,0,188367,0,0,0,0,0,2.1985e+06,0,0,375976,197683,53549,0,0,0,0,0,5.10148e+06,0,0,86222,0,381264,0,0,0,0,1.18756e+07,0,0,0,0,111819,0,0,0,0,0,0,0,0,4.67842e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SA.A_TSB_E,117105,3.42256e+06,0,246751,0,0,0,0,0,0,0,0,960937,0,0,93132,66816,0,0,0,0,0,0,2.34986e+06,0,0,0,0,233731,0,0,0,0,1.10244e+07,0,0,0,0,90520,0,0,0,0,0,0,0,0,3.21933e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,EC.B_TSB_A,119313,1.36455e+06,0,169495,0,0,0,0,0,0,0,0,0,0,0,54155,0,0,0,0,934149,151941,0,1.92851e+06,0,0,0,0,0,0,0,0,985092,2.3731e+06,0,0,0,0,0,0,265324,156741,82607,0,0,0,119773,0,0,410056,0,0,0,76521,0,0,0,289615,0,0,0,0,0,0,0,0,6.03256e+07
80,EC.B_TSB_B,128397,1.45114e+06,0,359499,0,0,0,0,0,0,0,0,0,0,0,54486,0,0,0,0,526119,185916,0,2.047e+06,0,0,0,0,0,0,0,0,617890,2.28252e+06,0,0,0,0,0,0,558260,299459,130593,0,0,0,235221,0,0,1.11834e+06,67510,0,0,125495,0,52814,0,749942,0,0,0,0,0,78672,0,0,1.0788e+08
81,EC.B_TSB_C,309004,5.65081e+06,0,875905,0,0,109170,0,0,0,310852,0,0,97651,0,180005,62168,0,0,0,809747,352271,0,6.49783e+06,0,0,92340,0,52741,58804,0,0,481673,8.92709e+06,0,0,0,97887,0,0,464549,362678,136519,0,0,0,360779,0,0,855428,69218,0,0,156776,0,0,0,479315,0,0,0,0,0,84426,0,0,7.21024e+07
82,EC.B_TSB_D,548442,3.58816e+06,0,637563,0,0,60834,0,0,0,483269,0,0,77088,0,114343,0,0,0,0,705595,244700,0,4.14133e+06,0,0,68147,0,85445,0,0,0,610883,6.38814e+06,0,0,0,80664,0,0,1.20103e+06,303087,162366,0,0,0,253951,0,0,2.40514e+06,169862,0,0,165001,0,84957,0,2.75081e+06,0,0,0,0,0,146349,0,0,8.52575e+07


In [33]:
df_t_bac[df_t_bac.index.name] = df_t_bac.index
print(df_t_bac)

Compounds       Ethyl Acetate      Ethanol  ...        Indole             NaN
Chemical_class          Ester      Alcohol  ...        Indole  Chemical_class
Unnamed: 2        888±8 (234)      932±8 (  ...  2445±10 (46)      Unnamed: 2
Unnamed: 3                 43           45  ...           117      Unnamed: 3
SA.A_TSB_A             465374  1.02772e+06  ...             0      SA.A_TSB_A
SA.A_TSB_B             193151  1.05097e+06  ...             0      SA.A_TSB_B
...                       ...          ...  ...           ...             ...
EC.B_TSB_A             119313  1.36455e+06  ...   6.03256e+07      EC.B_TSB_A
EC.B_TSB_B             128397  1.45114e+06  ...    1.0788e+08      EC.B_TSB_B
EC.B_TSB_C             309004  5.65081e+06  ...   7.21024e+07      EC.B_TSB_C
EC.B_TSB_D             548442  3.58816e+06  ...   8.52575e+07      EC.B_TSB_D
EC.B_TSB_E             339404  3.96293e+06  ...   9.71565e+07      EC.B_TSB_E

[87 rows x 68 columns]


In [16]:
display(list(df_t_bac.columns.values))

[' Ethyl Acetate',
 'Ethanol',
 'Propanoic acid, ethyl ester',
 '2-Pentanone',
 'Decane',
 'Methyl Isobutyl Ketone',
 'Amylene hydrate',
 'Butanoic acid, 2-methyl-, methyl ester',
 'Isobutyl acetate',
 'Methyl isovalerate',
 '1-Propanol',
 'Methyl thiolacetate',
 'Butanoic acid, 2-methyl-, ethyl ester',
 '2-Hexanone',
 'Ethyl isocyanide',
 '1-Propanol, 2-methyl-',
 '2-Pentanol, 2-methyl-',
 '2-Pentanol',
 '1-Butanol, 3-methyl-, acetate',
 '1 - Undecene',
 '1-Butanol',
 '2-Heptanone',
 'Dodecane',
 '1-Butanol, 3-methyl-',
 ' S-Methyl 3-methylbutanethioate',
 '2-Heptanone, 4,6-dimethyl-',
 ' 3-Buten-1-ol, 3-methyl-',
 'Thiocyanic acid, methyl ester',
 'Acetoin',
 '1-Pentanol, 2-methyl-',
 'Butanoic acid, 3-methyl-, 2-methylbutyl ester',
 '2-Heptanol, 4-methyl-',
 '2-Nonanone ',
 'Acetic acid',
 '2-Nonanol',
 'Pyrrole',
 '1H-Pyrrole, 2-methyl-',
 '1-Heptanol, 2,4-dimethyl-,',
 'Propanoic acid',
 'Propanoic acid, 2-methyl-',
 '1-Octanol',
 '2-Undecanone',
 'Benzoic acid, methyl ester',
 'B

In [11]:
df_t_bac['compounds'], df_t_bac['Bacteria'] = df_t_bac['Compounds'].str.split('.', 1).str
df_t_bac

KeyError: ignored



---


# Data visualizations

In [ ]:
pip install -U hiplot 

     |████████████████████████████████| 861 kB 33.8 MB/s 
     |████████████████████████████████| 788 kB 16.5 MB/s 
     |████████████████████████████████| 370 kB 48.2 MB/s 
     |████████████████████████████████| 357 kB 54.7 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
import hiplot as hip
hip.Experiment.from_dataframe(df_bac).display()

In [ ]:
sns.barplot(x="Compounds", y="Decane")

In [ ]:
import hiplot as hip
                data = [{'dropout':0.1, 'lr': 0.001, 'loss': 10.0, 'optimizer': 'SGD'},
                {'dropout':0.15, 'lr': 0.01, 'loss': 3.5, 'optimizer': 'Adam'},
                {'dropout':0.3, 'lr': 0.1, 'loss': 4.5, 'optimizer': 'Adam'}]
                hip.Experiment.from_iterable(data).display(force_full_width=True)
              

# TPOT 

In [ ]:
# Install tpot on the server
!pip install tpot

In [ ]:
# Import the tpot regressor
from tpot import TPOTRegressor

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Problem Description

The task is to predict the bacteria type the volotiles were measured from and perhaps even predict their culture medium. 

### Data

The data includes 3 bacteria types over 84 stems that were measured in total and 67 volatile compounds that were detected at least in one measurement. 

In [ ]:
from tpot import TPOTClassifier

In [ ]:
# Convert to numpy arrays (using transposed data set)
t_bac = np.array(df_t_bac)
#testing_features = np.array(test_features)

# Sklearn wants the labels as one-dimensional vectors
t_bac_targets = np.array(df_t_bac).reshape((-1,))
#training_targets = np.array(train_labels).reshape((-1,))
#testing_targets = np.array(test_labels).reshape((-1,))

In [ ]:
# Create a tpot object with a few parameters
tpot = TPOTRegressor(scoring = 'neg_mean_absolute_error', 
                    max_time_mins = 480, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 5)

In [ ]:
# Fit the tpot model on the training data
tpot.fit(t_bac)

TypeError: ignored